In [45]:
import requests

In [46]:
def body(url):
    # This function will print the content of the body.
    response = requests.get(url)
    html_content = response.text
    texts = []
    start_index = html_content.find("<body")

    if start_index != -1:
        start_index = html_content.find(">", start_index) + 1
        while start_index != -1:
            end_index = html_content.find("<", start_index)
            if end_index != -1:
                text = html_content[start_index:end_index].strip()
                if text: #text.isalnum()
                    texts.append(text)
                start_index = html_content.find(">", end_index) + 1
            else:
                break
    # This will remove the unwanted things from the texts and add to the result.
    result = ''
    for i in texts:
        if (i[0] not in ['.', ',','', ' ', '(', ')', '[', ']', '{', '}', '@', "'", '&', '#', '^']):
            if i[0:3] not in ['-->', 'htt', 'jQu']:
                # if i.isalnum():
                    result = result + i + ' '
            
    return result

In [55]:
x = body('https://en.wikipedia.org/wiki/Machine_learning')

In [56]:
# def tokenize_text(x):
#     words = re.findall(r'\b\w+\b', x.lower())
#     return words
# x = tokenize_text(x)

In [65]:
# Making n-grams
def generate_ngrams(text, n_sizes):
    ngrams = {}
    for n in n_sizes:
        ngrams[n] = []
        for i in range(len(text) - n + 1):
            ngrams[n].append(text[i:i+n])
    return ngrams

In [66]:
text = x.split()
n_sizes = [1, 2, 3, 4, 5]

#result is a dict
result = generate_ngrams(text, n_sizes)
for n in n_sizes:
    print(f"{n}-grams:", result[n])

1-grams: [['Jump'], ['to'], ['content'], ['Main'], ['menu'], ['Main'], ['menu'], ['move'], ['to'], ['sidebar'], ['hide'], ['Navigation'], ['Main'], ['page'], ['Contents'], ['Current'], ['events'], ['Random'], ['article'], ['About'], ['Wikipedia'], ['Contact'], ['us'], ['Donate'], ['Contribute'], ['Help'], ['Learn'], ['to'], ['edit'], ['Community'], ['portal'], ['Recent'], ['changes'], ['Upload'], ['file'], ['Languages'], ['Language'], ['links'], ['are'], ['at'], ['the'], ['top'], ['of'], ['the'], ['page.'], ['Search'], ['Search'], ['Create'], ['account'], ['Log'], ['in'], ['Personal'], ['tools'], ['Create'], ['account'], ['Log'], ['in'], ['Pages'], ['for'], ['logged'], ['out'], ['editors'], ['learn'], ['more'], ['Contributions'], ['Talk'], ['Contents'], ['move'], ['to'], ['sidebar'], ['hide'], ['1'], ['History'], ['and'], ['relationships'], ['to'], ['other'], ['fields'], ['Toggle'], ['History'], ['and'], ['relationships'], ['to'], ['other'], ['fields'], ['subsection'], ['1.1'], ['Artif

In [67]:
import requests
from bs4 import BeautifulSoup
import re

def fetch_text_from_url(url):
    # Fetch HTML content from the URL
    response = requests.get(url)
    if response.status_code == 200:
        # Extract text from HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        body_text = soup.get_text()
        return body_text
    else:
        print("Failed to fetch content from URL:", url)
        return None

def tokenize_text(text):
    # Tokenize the text into words
    words = re.findall(r'\b\w+\b', text.lower())
    return words

def compute_word_frequency(words):
    # Count the frequency of each word
    word_freq = {}
    for word in words:
        word_freq[word] = word_freq.get(word, 0) + 1
    return word_freq

def polynomial_hash(word):
    # Implement polynomial rolling hash function
    p = 53
    m = 2**64
    hash_val = 0
    for i, char in enumerate(word):
        hash_val += ord(char) * (p ** i)
    return hash_val % m

def compute_simhash(word_hashes):
    # Combine word hashes into a single Simhash value
    simhash = 0
    for word_hash in word_hashes:
        simhash ^= word_hash
    return simhash

def compare_simhashes(simhash1, simhash2):
    # Compare two Simhash values bitwise
    common_bits = bin(simhash1 ^ simhash2).count('0')
    return common_bits

if __name__ == "__main__":
    url1 = input("Enter URL 1: ")
    url2 = input("Enter URL 2: ")
    
    # Fetch text from URLs
    text1 = fetch_text_from_url(url1)
    text2 = fetch_text_from_url(url2)
    
    if text1 and text2:
        # Tokenize text
        words1 = tokenize_text(text1)
        words2 = tokenize_text(text2)
        
        # Compute word frequency
        word_freq1 = compute_word_frequency(words1)
        word_freq2 = compute_word_frequency(words2)
        
        # Compute word hashes
        word_hashes1 = [polynomial_hash(word) for word in words1]
        word_hashes2 = [polynomial_hash(word) for word in words2]
        
        # Compute Simhash
        simhash1 = compute_simhash(word_hashes1)
        simhash2 = compute_simhash(word_hashes2)
        
        # Compare Simhashes
        common_bits = compare_simhashes(simhash1, simhash2)
        print("Number of common bits in Simhashes:", common_bits)
    else:
        print("Failed to fetch content from one or both URLs.")


MissingSchema: Invalid URL '1': No schema supplied. Perhaps you meant http://1?

In [ ]:
p = 53
m = 2**64
hash_val = 0
for i, char in enumerate(word):
    hash_val += ord(char) * (p ** i)
    
result = hash_val % m

In [ ]:
def compute_simhash(word_hashes):
    # Combine word hashes into a single Simhash value
    simhash = 0
    for word_hash in word_hashes:
        simhash ^= word_hash
    return simhash


In [ ]:
def compare_simhashes(simhash1, simhash2):
    # Compare two Simhash values bitwise
    common_bits = bin(simhash1 ^ simhash2).count('0')
    return common_bits